In [21]:
import tkinter as tk
from tkinter import messagebox
import hashlib
import os
import json

CREDENTIALS_FILE = "credentials.json"
TASKS_FILE = "tasks.json"

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

if not os.path.exists(CREDENTIALS_FILE):
    with open(CREDENTIALS_FILE, 'w') as f:
        json.dump({}, f)

if not os.path.exists(TASKS_FILE):
    with open(TASKS_FILE, 'w') as f:
        json.dump({}, f)

class TaskManager:
    def __init__(self, root):
        self.root = root
        self.root.title("Task Manager")
        self.root.configure(bg="pink")
        self.root.geometry("400x600")
        self.logged_in_user = None

        self.show_login_screen()

    def show_login_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Task Manager", font=("Arial", 20), bg="pink").pack(pady=20)
        tk.Label(self.root, text="Username", bg="pink").pack()
        self.username_entry = tk.Entry(self.root)
        self.username_entry.pack()

        tk.Label(self.root, text="Password", bg="pink").pack()
        self.password_entry = tk.Entry(self.root, show="*")
        self.password_entry.pack()

        tk.Button(self.root, text="Login", command=self.login).pack(pady=5)
        tk.Button(self.root, text="Register", command=self.register).pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        with open(CREDENTIALS_FILE, 'r') as f:
            credentials = json.load(f)

        if username in credentials and credentials[username] == hash_password(password):
            self.logged_in_user = username
            self.show_task_manager()
        else:
            messagebox.showerror("Error", "Invalid credentials!")

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        if not username or not password:
            messagebox.showerror("Error", "Username and password cannot be empty!")
            return

        with open(CREDENTIALS_FILE, 'r') as f:
            credentials = json.load(f)

        if username in credentials:
            messagebox.showerror("Error", "Username already exists!")
        else:
            credentials[username] = hash_password(password)
            with open(CREDENTIALS_FILE, 'w') as f:
                json.dump(credentials, f)
            messagebox.showinfo("Success", "Registration successful!")

    def show_task_manager(self):
        self.clear_screen()

        tk.Label(self.root, text=f"Welcome, {self.logged_in_user}", font=("Arial", 16), bg="pink").pack(pady=10)
        tk.Button(self.root, text="Add Task", command=self.add_task_screen).pack(fill=tk.X, pady=5)
        tk.Button(self.root, text="View Tasks", command=self.view_tasks_screen).pack(fill=tk.X, pady=5)
        tk.Button(self.root, text="Logout", command=self.logout).pack(fill=tk.X, pady=5)

    def add_task_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Add Task", font=("Arial", 16), bg="pink").pack(pady=10)
        tk.Label(self.root, text="Task Description", bg="pink").pack()
        self.task_description_entry = tk.Entry(self.root)
        self.task_description_entry.pack()

        tk.Button(self.root, text="Add", command=self.add_task).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_task_manager).pack()

    def add_task(self):
        task_description = self.task_description_entry.get()
        if not task_description:
            messagebox.showerror("Error", "Task description cannot be empty!")
            return

        with open(TASKS_FILE, 'r') as f:
            tasks = json.load(f)

        if self.logged_in_user not in tasks:
            tasks[self.logged_in_user] = []

        task_id = len(tasks[self.logged_in_user]) + 1
        tasks[self.logged_in_user].append({"id": task_id, "description": task_description, "status": "Pending"})

        with open(TASKS_FILE, 'w') as f:
            json.dump(tasks, f)

        messagebox.showinfo("Success", "Task added successfully!")
        self.show_task_manager()

    def view_tasks_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Your Tasks", font=("Arial", 16), bg="pink").pack(pady=10)

        with open(TASKS_FILE, 'r') as f:
            tasks = json.load(f)

        user_tasks = tasks.get(self.logged_in_user, [])
        for task in user_tasks:
            tk.Label(self.root, text=f"ID: {task['id']} | {task['description']} | {task['status']}", bg="pink").pack()

        tk.Button(self.root, text="Mark Task as Completed", command=self.mark_task_completed_screen).pack(pady=5)
        tk.Button(self.root, text="Delete Task", command=self.delete_task_screen).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_task_manager).pack()

    def mark_task_completed_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Mark Task as Completed", font=("Arial", 16), bg="pink").pack(pady=10)
        tk.Label(self.root, text="Enter Task ID", bg="pink").pack()
        self.task_id_entry = tk.Entry(self.root)
        self.task_id_entry.pack()

        tk.Button(self.root, text="Mark Completed", command=self.mark_task_completed).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.view_tasks_screen).pack()

    def mark_task_completed(self):
        task_id = self.task_id_entry.get()
        if not task_id.isdigit():
            messagebox.showerror("Error", "Invalid Task ID!")
            return

        with open(TASKS_FILE, 'r') as f:
            tasks = json.load(f)

        user_tasks = tasks.get(self.logged_in_user, [])
        for task in user_tasks:
            if task['id'] == int(task_id):
                task['status'] = "Completed"
                with open(TASKS_FILE, 'w') as f:
                    json.dump(tasks, f)
                messagebox.showinfo("Success", "Task marked as completed!")
                self.view_tasks_screen()
                return

        messagebox.showerror("Error", "Task not found!")

    def delete_task_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Delete Task", font=("Arial", 16), bg="pink").pack(pady=10)
        tk.Label(self.root, text="Enter Task ID", bg="pink").pack()
        self.task_id_entry = tk.Entry(self.root)
        self.task_id_entry.pack()

        tk.Button(self.root, text="Delete", command=self.delete_task).pack(pady=5)
        tk.Button(self.root, text="Back", command=self.view_tasks_screen).pack()

    def delete_task(self):
        task_id = self.task_id_entry.get()
        if not task_id.isdigit():
            messagebox.showerror("Error", "Invalid Task ID!")
            return

        with open(TASKS_FILE, 'r') as f:
            tasks = json.load(f)

        user_tasks = tasks.get(self.logged_in_user, [])
        for task in user_tasks:
            if task['id'] == int(task_id):
                user_tasks.remove(task)
                with open(TASKS_FILE, 'w') as f:
                    json.dump(tasks, f)
                messagebox.showinfo("Success", "Task deleted!")
                self.view_tasks_screen()
                return

        messagebox.showerror("Error", "Task not found!")

    def logout(self):
        self.logged_in_user = None
        self.show_login_screen()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()


root = tk.Tk()
app = TaskManager(root)
root.mainloop()
